In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
print(os.listdir("../input/aptos2019-blindness-detection"))
print(os.listdir("../input/aptocsubbmition"))
print(os.listdir("../input/aptos20192"))

['aptos2019-blindness-detection', 'aptos20192', 'aptocsubbmition']
['test.csv', 'test_images', 'train.csv', 'sample_submission.csv', 'train_images']
['mysubmisson.csv']
['mean_v4.npy', 'model.model', 'label_dict_v4.pickle']


In [2]:
import sys
import csv
from collections import defaultdict
import six
import time 

import warnings 
import io
import cgi
import cgitb
 
import sys
import codecs
import locale 
cgitb.enable()
import cupy as np
import pandas as pd
import chainer
from chainer import cuda, Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions
import pickle 
import re
import json
import cv2
import io
import scipy.misc
import base64
import numpy 
from imageio import imread
from PIL import Image

import chainer
import chainer.functions as F
from chainer import initializers
import chainer.links as L
import random




In [3]:
class GoogLeNetBN(chainer.Chain):

    """New GoogLeNet of BatchNormalization version."""

    insize = 224

    def __init__(self):
        super(GoogLeNetBN, self).__init__(
            conv1=L.Convolution2D(None, 64, 7, stride=2, pad=3, nobias=True),
            norm1=L.BatchNormalization(64),
            conv2=L.Convolution2D(None, 192, 3, pad=1, nobias=True),
            norm2=L.BatchNormalization(192),
            inc3a=L.InceptionBN(None, 64, 64, 64, 64, 96, 'avg', 32),
            inc3b=L.InceptionBN(None, 64, 64, 96, 64, 96, 'avg', 64),
            inc3c=L.InceptionBN(None, 0, 128, 160, 64, 96, 'max', stride=2),
            inc4a=L.InceptionBN(None, 224, 64, 96, 96, 128, 'avg', 128),
            inc4b=L.InceptionBN(None, 192, 96, 128, 96, 128, 'avg', 128),
            inc4c=L.InceptionBN(None, 128, 128, 160, 128, 160, 'avg', 128),
            inc4d=L.InceptionBN(None, 64, 128, 192, 160, 192, 'avg', 128),
            inc4e=L.InceptionBN(None, 0, 128, 192, 192, 256, 'max', stride=2),
            inc5a=L.InceptionBN(None, 352, 192, 320, 160, 224, 'avg', 128),
            inc5b=L.InceptionBN(None, 352, 192, 320, 192, 224, 'max', 128),
            out=L.Linear(None, 1000),

            conva=L.Convolution2D(None, 128, 1, nobias=True),
            norma=L.BatchNormalization(128),
            lina=L.Linear(None, 1024, nobias=True),
            norma2=L.BatchNormalization(1024),
            outa=L.Linear(None, 1000),

            convb=L.Convolution2D(None, 128, 1, nobias=True),
            normb=L.BatchNormalization(128),
            linb=L.Linear(None, 1024, nobias=True),
            normb2=L.BatchNormalization(1024),
            outb=L.Linear(None, 1000),
        )
        self._train = True

    @property
    def train(self):
        return self._train

    @train.setter
    def train(self, value):
        self._train = value
        self.inc3a.train = value
        self.inc3b.train = value
        self.inc3c.train = value
        self.inc4a.train = value
        self.inc4b.train = value
        self.inc4c.train = value
        self.inc4d.train = value
        self.inc4e.train = value
        self.inc5a.train = value
        self.inc5b.train = value

    def __call__(self, x, t):

        h = F.max_pooling_2d(
            F.relu(self.norm1(self.conv1(x))), 3, stride=2, pad=1)
        h = F.max_pooling_2d(
            F.relu(self.norm2(self.conv2(h))), 3, stride=2, pad=1)

        h = self.inc3a(h)
        h = self.inc3b(h)
        h = self.inc3c(h)
        h = self.inc4a(h)

        a = F.average_pooling_2d(h, 5, stride=3)
        a = F.relu(self.norma(self.conva(a)))
        a = F.relu(self.norma2(self.lina(a)))
        a = self.outa(a)
        loss1 = F.softmax_cross_entropy(a, t)

        h = self.inc4b(h)
        h = self.inc4c(h)
        h = self.inc4d(h)

        b = F.average_pooling_2d(h, 5, stride=3)
        b = F.relu(self.normb(self.convb(b)))
        b = F.relu(self.normb2(self.linb(b)))
        b = self.outb(b)
        loss2 = F.softmax_cross_entropy(b, t)

        h = self.inc4e(h)
        h = self.inc5a(h)
        h = F.average_pooling_2d(self.inc5b(h), 7)
        h = self.out(h)
        loss3 = F.softmax_cross_entropy(h, t)

        loss = 0.3 * (loss1 + loss2) + loss3
        accuracy = F.accuracy(h, t)

        chainer.report({
            'loss': loss,
            'loss1': loss1,
            'loss2': loss2,
            'loss3': loss3,
            'accuracy': accuracy,
        }, self)
        return loss

    def predict(self, x):
        with chainer.no_backprop_mode(),chainer.using_config('train',False):
            h = F.max_pooling_2d(
                F.relu(self.norm1(self.conv1(x))), 3, stride=2, pad=1)
            h = F.max_pooling_2d(
                F.relu(self.norm2(self.conv2(h))), 3, stride=2, pad=1)

            h = self.inc3a(h)
            h = self.inc3b(h)
            h = self.inc3c(h)
            h = self.inc4a(h)

            # a = F.average_pooling_2d(h, 5, stride=3)
            # a = F.relu(self.norma(self.conva(a)))
            # a = F.relu(self.norma2(self.lina(a)))
            # a = self.outa(a)
            # loss1 = F.softmax_cross_entropy(a, t)

            h = self.inc4b(h)
            h = self.inc4c(h)
            h = self.inc4d(h)

            # b = F.average_pooling_2d(h, 5, stride=3)
            # b = F.relu(self.normb(self.convb(b)))
            # b = F.relu(self.normb2(self.linb(b)))
            # b = self.outb(b)
            # loss2 = F.softmax_cross_entropy(b, t)

            h = self.inc4e(h)
            h = self.inc5a(h)
            h = F.average_pooling_2d(self.inc5b(h), 7)
            h = self.out(h)

            return F.softmax(h)

In [4]:
UNK = 0
EOS = 1
class load_model():
    def main():
        model = GoogLeNetBN()
        serializers.load_npz("../input/aptos20192/model.model", model)
        return model 
class model_define():
    start = time.time()
    print("****************************************************************")
    print("Load models")
    
    model1 = load_model.main()    
    chainer.serializers.save_npz('model2.model', model1)
    
    print("Model defined elapsed time(経過時間) : {0}".format(time.time() - start) + "[sec]")

def pred():
    start = time.time() 
    submit_list = [['id_code','diagnosis']]
    df_test = pd.read_csv('../input/aptos2019-blindness-detection/test.csv').values.tolist()
    with open('../input/aptos20192/label_dict_v4.pickle','rb') as f:
        categories = pickle.load(f)
    mean_image =  np.load('../input/aptos20192/mean_v4.npy') 
    model = model_define.model1
    cropwidth = 256 - model.insize
    size = (256, 256) 
    for item in df_test:
        img = imread('../input/aptos2019-blindness-detection/test_images/' + item[0] + '.png') 
        #Create Half Size Image
        #halfImg = cv2.resize(img, (orgHeight / 2, orgWidth / 2))
        image = cv2.resize(img, size)
        image = np.asarray(image).transpose(2, 0, 1)

        
        top = random.randint(0, cropwidth - 1)
        left = random.randint(0, cropwidth - 1)
        bottom = model.insize + top
        right = model.insize + left
        image = image[:, top:bottom, left:right].astype(np.float32)
        image -= mean_image[:, top:bottom, left:right]
        image /= 255
            
        model.to_gpu()  

        x = np.ndarray(
                (1, 3, model.insize, model.insize), dtype=np.float32)
        x[0]=image  
        x = chainer.Variable(x)
        score = model.predict(x) 


        top_k =10
        prediction = list(zip(score.data[0].tolist(), [v for k,v in categories.items()]))
        prediction= sorted(prediction,reverse=True)
        submit_list.append([item[0],prediction[0][1]]) 
 
    with open('submission.csv', 'w') as f:
        writer = csv.writer(f, lineterminator='\n') # 改行コード（\n）を指定しておく 
        writer.writerows(submit_list) # 2次元配列も書き込める  
    print("end:{0}".format(time.time()-start) +'[sec]')
if __name__ == "__main__":
    model_define()
    pred()

****************************************************************
Load models
Model defined elapsed time(経過時間) : 5.325917482376099[sec]
end:130.75533962249756[sec]
